In [79]:
def union2(p1,p2):
    '''This def will return the union of two binary searches''' 
    L3 = [max(l1, l2) for l1, l2 in zip(p1, p2)]
    return(L3)
def intersection2(p1,p2):
    '''This def will return the intersection of two binary searches'''
    L3 = [min(l1, l2) for l1, l2 in zip(p1, p2)]
    return(L3)
def tokenize(search):
    '''This def tokenize, lower csae, and stem all the strings'''
    search_token = word_tokenize(search) #Tokenize
    search_token = [i.lower() for i in search_token] # Lower Case
    search_token = [porter.stem(word) for word in search_token] #Stem
    return(search_token)
def extractkeys(tokens):
    '''This token will extract the operation sumbols and match the keys of each tokens with our binary matrix'''
    unionwords = ['or','and']
    operations = [word for word in unionwords if word in tokens] #operation
    keys = [word for word in tokens if word not in unionwords and word in [*mydict.keys()]] #keys
    return(operations, keys)
def operation(p1,p2,operation):
    '''This def does the union and intersection, given the binary data, and the operation types (or, and)'''
    out=list()
    if operation == ['or']:
        out=union2(p1,p2)
    elif operation == ['and']:
        out=intersection2(p1,p2)
    return(out)

In [82]:
help(prepro)

Help on function prepro in module __main__:

prepro(docs='docs.txt')
    This is the preprocessing module, whitch given a txt document that each new line is one document, and each line starts with the documentID. 
    This Def will return the term-document incidence matrix



In [81]:
def prepro(docs = "docs.txt"):
    ''' This is the preprocessing module, whitch given a txt document that each new line is one document, and each line starts with the documentID. 
    This Def will return the term-document incidence matrix'''
    infile = open(docs,'r')
    docs=infile.readlines()
    docs=[i.strip() for i in docs] #removing the \n
    docs=[i.lower() for i in docs] #Lower Case

    tokenized = [word_tokenize(docs[i]) for i in range(len(docs))]
    [i.pop(0) for i in tokenized] # Removing the ID

    porter = PorterStemmer()
    stemmed=[]
    for i in range(len(docs)):
        stem = [porter.stem(word) for word in tokenized[i]]
        stemmed.append(stem)

    tokens = set(x for l in stemmed for x in l)
    mydict = dict()
    for el in tokens:
        mydict[el]=[stemmed[i].count(el) for i in range(len(docs))]
    return(mydict)

In [104]:
def process(search,mydict):
    '''This def, takes the operation search (i.e. patiens and drugs), and term-document incidence matrix and finds the search result. The search results are shown in a binary list, which each value of this arry represent document (starting from the 1st document to the last.)'''
    inside=re.findall('\((.*?)\)',search)
    outside = re.findall('\)(.*?)\(',search)
    search_token = tokenize(search)
    operations, search_keys = extractkeys(search_token)

    if len(search_keys)==1:
        result['operat']=mydict[search_keys[0]]
    if len(search_keys)==2:  
        result['operat']=operation(mydict[search_keys[0]],mydict[search_keys[1]],operations)

    if len(search_keys) > 2:
        for i in range(len(inside)):
            tokens = tokenize(inside[i])
            operations_inside, keys_inside = extractkeys(tokens)
            result[i]=operation(mydict[keys_inside[0]],mydict[keys_inside[1]],operations_inside)

        if outside:
            operations_outside = [tokenize(i) for i in outside]
            result['operat'] = operation(result[0], result[1],operations_outside[0])
            if len(operations_outside) > 1:
                print (outside)
                print (operations_outside)
                for j in range(1,len(operations_outside)):
                    result['operat'] = operation(result['operat'], result[j+1],operations_outside[j])
        else:
            result['operat']=result[0]
        #First:
        if search_token[0]!='(':
            result['operat']=operation(result['operat'], mydict[search_token[0]],[search_token[1]])
            
        #Last:    
        if search_token[-1]!=')':
            result['operat']=operation(result['operat'], mydict[search_token[-1]],[search_token[-2]])
    return(result['operat'])

In [147]:
def process2(search,mydict):
    '''This def, takes the operation search (i.e. patiens and drugs), and term-document incidence matrix and finds the search result. The search results are shown in a binary list, which each value of this arry represent document (starting from the 1st document to the last.)'''
    inside=re.findall('\((.*?)\)',search)
    outside = re.findall('\)(.*?)\(',search)
    search_token = tokenize(search)
    operations, search_keys = extractkeys(search_token)

    if len(search_keys)==1:
        result['operat']=mydict[search_keys[0]]
    if len(search_keys)==2:  
        result['operat']=operation(mydict[search_keys[0]],mydict[search_keys[1]],operations)

    if len(search_keys) > 2:
        for i in range(len(inside)):
            tokens = tokenize(inside[i])
            operations_inside, keys_inside = extractkeys(tokens)
            result[i]=operation(mydict[keys_inside[0]],mydict[keys_inside[1]],operations_inside)
        
        #First:
        if search_token[0]!='(':
            result['operat']=operation(result[0], mydict[search_token[0]],[search_token[1]])
        else:
            result['operat']=result[0]
        print(result['operat'])
        #Middle:
        
        if outside:
            operations_outside = [tokenize(i) for i in outside]
            for j in range(0,len(operations_outside)):
                result['operat'] = operation(result['operat'], result[j+1],operations_outside[j])
        #Last:    
        if search_token[-1]!=')':
            result['operat']=operation(result['operat'], mydict[search_token[-1]],[search_token[-2]])
    return(result['operat'])

In [148]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer


unionwords = ['or','and']
result=dict()
# search = 'treatment  or (schizophrenia and drugs) and (treatment and drugs) and drugs'
# search = '(treatment and drugs) and drugs'
def main():
    search=input("give the operation:")
    mydict = prepro()
    result = process2(search,mydict)
    return(result)

In [149]:
main()
#treatment  or (schizophrenia and drugs) or (treatment and drugs) and drugs

give the operation:treatment  or (schizophrenia and drugs) or (treatment and drugs) and drugs
[1, 1, 0, 1]


[1, 0, 0, 1]

In [ ]:
operation(p1,p2,operation)

In [584]:
# unionwords = ['or','and']
# operations = [word for word in unionwords if word in search_token  ]
# keys = [word for word in search_token if word not in unionwords and word in [*mydict.keys()]]

'doc2 new approach for treatment of schizophrenia'

['doc1', 'doc2', 'doc3', 'doc4']

In [587]:

stemmed

[['breakthrough', 'drug', 'for', 'schizophrenia'],
 ['new', 'approach', 'for', 'treatment', 'of', 'schizophrenia'],
 ['new', 'hope', 'for', 'schizophrenia', 'patient'],
 ['new', 'schizophrenia', 'drug']]

In [588]:
tokens

{'approach',
 'breakthrough',
 'drug',
 'for',
 'hope',
 'new',
 'of',
 'patient',
 'schizophrenia',
 'treatment'}

In [589]:
treatment  or (schizophrenia and drugs) and (treatment and drugs) and drugs

{'approach': [0, 1, 0, 0],
 'breakthrough': [1, 0, 0, 0],
 'drug': [1, 0, 0, 1],
 'for': [1, 1, 1, 0],
 'hope': [0, 0, 1, 0],
 'new': [0, 1, 1, 1],
 'of': [0, 1, 0, 0],
 'patient': [0, 0, 1, 0],
 'schizophrenia': [1, 1, 1, 1],
 'treatment': [0, 1, 0, 0]}

In [590]:
mydict = dict((el,[stemmed[0].count(el),stemmed[1].count(el),stemmed[2].count(el),stemmed[3].count(el)]) for el in tokens)

In [575]:
mydict['breakthrough'][0]

1

In [576]:
mydict['for']

[1, 1, 1, 0]

In [9]:
def intersection(p1,p2):
    mydict['intersection']=mydict[p1].copy()
    for i in range(len(docs)):
        if mydict[p1][i] == mydict[p2][i] == 1:
            mydict['intersection'][i]=1
        else:
            mydict['intersection'][i]=0
    return(mydict['intersection'])

In [10]:
intersection('schizophrenia','drug')

[1, 0, 0, 1]

In [11]:
def union(p1,p2):
    mydict['union']=mydict[p1].copy()
    for i in range(len(docs)):
        if mydict[p2][i] == 1:
            mydict['union'][i]=1
    return(mydict['union'])

In [12]:
union('schizophrenia','drug')

[1, 1, 1, 1]

In [13]:
mydict['schizophrenia']

[1, 1, 1, 1]

In [14]:
mydict['drug']

[1, 0, 0, 1]

In [15]:
mydict['treatment']

[0, 1, 0, 0]

In [16]:
mydict['schizophrenia']

[1, 1, 1, 1]

In [17]:
def unioninter(p1,p2,p3):
    union(p1,p2)
    intersection('union',p3)
    return(mydict['intersection'])

In [18]:
unioninter('drug' , 'treatment' , 'schizophrenia')

[1, 1, 0, 1]

In [19]:
search = input("what are you looking for? ")
# drug OR treatment AND schizophrenia

what are you looking for? a;d


In [460]:
search = 'treatment  or (schizophrenia and drugs) and (treatment and drugs) and drugs'

In [461]:
# inside = search[search.find("(")+1:search.find(")")]
import re
inside=re.findall('\((.*?)\)',search)
inside

['schizophrenia and drugs', 'treatment and drugs']

In [462]:
left=re.match('(.*?)\(',search)
outside = re.findall('\)(.*?)\(',search)
outside
# [tokenize(i) for i in outside]
if outside:
    print("yes")

yes


In [369]:
search_token = word_tokenize(search)
search_token=[i.lower() for i in search_token] #Lower Case
search_token = [porter.stem(word) for word in search_token]

In [393]:
# left_parant = [ word for word in search_token[0:2] if word in ['(',')']]


False

In [101]:
def tokenize(search):
    search_token = word_tokenize(search) #Tokenize
    search_token = [i.lower() for i in search_token] # Lower Case
    search_token = [porter.stem(word) for word in search_token] #Stem
    return(search_token)
def extractkeys(tokens):
    unionwords = ['or','and']
    operations = [word for word in unionwords if word in tokens] #operation
    keys = [word for word in tokens if word not in unionwords and word in [*mydict.keys()]] #keys
    return(operations, keys)
    

In [396]:
if search_token[0]!='(':
    print('yes')

yes


In [498]:
unionwords = ['or','and']
operations = [word for word in unionwords if word in search_token  ]
keys = [word for word in search_token if word not in unionwords and word in [*mydict.keys()]]


operations
keys

['treatment', 'schizophrenia', 'drug', 'treatment', 'drug', 'drug']

In [499]:
a = tokenize(sina[0])
# print(a)
# keys = [word for word in a if word not in unionwords and word in [*mydict.keys()]]
# print(keys)
c,d =extractkeys(a)
c

['and']

In [539]:
docs=dict()
if len(keys)==1:
    docs['one_word']=mydict[keys[0]]
if len(keys)==2 and operations == ['or']:
    docs['one_union']=union(keys[0],keys[1])
if len(keys)==2 and operations == ['and']:
    docs['one_inter']=intersection(keys[0],keys[1])

if len(keys) > 2:
    for i in range(len(inside)):
        tokens = tokenize(inside[i])
        operations_inside, keys_inside = extractkeys(tokens)
        if operations_inside == ['or']:
            docs[i]=union(keys_inside[0],keys_inside[1])
        if operations_inside == ['and']:
            docs[i]=union(keys_inside[0],keys_inside[1])

    if outside:
        operations_outside = [tokenize(i) for i in outside]
        print(operations_outside[0])
        print(docs[0], docs[1])
        docs['operat'] = operation(docs[0], docs[1],operations_outside[0])
        print(docs['operat'])
        if len(operations_outside) > 1:
            for j in range(1,len(operations_outside)):
                docs['operat'] = operation(docs['operat'], docs[j+1],operations_outside[j])     
            
            #First:
    if search_token[0]!='(':
        print(docs['operat'],mydict[search_token[0]],search_token[1])
        
        docs['operat']=operation(docs['operat'], mydict[search_token[0]],[search_token[1]])
        print(docs['operat'])
        
#         docs['first']=union2(search_token[0],docs[0])
#     elif search_token[0]!='(' and search_token[1] == ['and']:
#         docs['first']=intersection2(search_token[0],docs[0])

            #Last:    
    if search_token[-1]!=')':
        print(docs['operat'],mydict[search_token[-1]],[search_token[-1]],[search_token[-2]])
        docs['operat']=operation(docs['operat'], mydict[search_token[-1]],[search_token[-2]])
        print(docs['operat'])
        
#     if search_token[-1]!=')' and search_token[-2] == ['or']:
#         docs['last']=union2(search_token[-1],docs[i])
#     elif search_token[-1]!='(' and search_token[-2] == ['and']:
#         docs['last']=intersection2(search_token[-1],docs[i])
        
    
        
#         if 'first' in docs.keys():
#             docs['operat'] = operation(docs['first'], docs[1],operations_outside[0])
#         else:
#             docs['operat'] = docs['first']
            
    


['and']
[1, 1, 1, 1] [1, 1, 0, 0]
[1, 1, 0, 0]
[1, 1, 0, 0] [0, 1, 0, 0] or
[1, 1, 0, 0]
[1, 1, 0, 0] [1, 0, 0, 1] ['drug'] ['and']
[1, 0, 0, 0]


In [525]:
mydict[search_token[0]]

[0, 1, 0, 0]

In [533]:
docs

{0: [1, 1, 1, 1], 1: [1, 1, 0, 0], 'operat': [1, 0, 0, 0]}

In [496]:
operations_outside=[1,2,3]
range(len(operations_outside))

range(0, 3)

In [474]:
if 0 in docs.keys():
    print('yes')

yes


In [511]:
docs['operat']

[]

In [505]:
def operation(p1,p2,operation):
    out=list()
    if operation == ['or']:
        out=union2(p1,p2)
    elif operation == ['and']:
        out=intersection2(p1,p2)
    return(out)

In [507]:
operation([0,0,0,0],[1,2,0,2],['and'])

[0, 0, 0, 0]

In [408]:
docs['one_inter']

[0, 1, 0, 0]